## Testing OpenCV's DeepLearning Module with pre-trained GoogleNet
###### v20180822
* Implementation with pretrained Standard GoogleNet

In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
# import the necessary packages
import numpy as np
import time
import cv2
from pathlib import Path

In [14]:
# Set PATH if other than home directory
PATH = Path('')

In [25]:
# load the input image from disk
image = cv2.imread('b.jpg')
print(image.mean())

211.88758157389634


In [16]:
# load the class labels from disk
rows = open("synset_words_raw.txt").read().strip().split("\n")
classes = [r[r.find(" ") + 1:].split(",")[0] for r in rows]

In [31]:
## Input Normalization
# - resizing to 224x224 pixels while
# - mean subtraction (104, 117, 123)
blob = cv2.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))
print(blob.shape)

(1, 3, 224, 224)


In [32]:
# load our serialized model from disk
net = cv2.dnn.readNetFromCaffe("deployGN.prototxt", "bvlc_googlenet.caffemodel")
print("[INFO] model is loaded")

[INFO] model is loaded


In [33]:
# set the blob as input to the network and perform a forward-pass to
# obtain our output classification
net.setInput(blob)
start = time.time()
preds = net.forward()
end = time.time()
print("[INFO] classification took {:.5} seconds".format(end - start))

[INFO] classification took 0.093734 seconds


In [34]:
# sort the indexes of the probabilities in descending order
# and grab the top-5 predictions
idxs = np.argsort(preds[0])[::-1][:5]
print(idxs)

[309 308 749 319 320]


In [35]:
# loop over the top-5 predictions and display them
for (i, idx) in enumerate(idxs):
    # draw the top prediction on the input image
    if i == 0:
        text = "Label: {}, {:.2f}%".format(classes[idx],
                                           preds[0][idx] * 100)
        cv2.putText(image, text, (5, 25), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 0, 255), 2)
        
    # display the predicted label + associated probability to the
    # console
    print("[INFO] {}. label: {}, probability: {:.5}".format(i + 1,
                                                            classes[idx], preds[0][idx]))

[INFO] 1. label: bee, probability: 0.61671
[INFO] 2. label: fly, probability: 0.20969
[INFO] 3. label: quill, probability: 0.059565
[INFO] 4. label: dragonfly, probability: 0.049366
[INFO] 5. label: damselfly, probability: 0.029728


In [36]:
cv2.imshow('image',image)

In [37]:
cv2.waitKey(0)

-1